This data contains 961 instances of masses detected in mammograms, and contains the following attributes:

BI-RADS assessment: 1 to 5 (ordinal) Age: patient's age in years (integer) Shape: mass shape: round=1 oval=2 lobular=3 irregular=4 (nominal) Margin: mass margin: circumscribed=1 microlobulated=2 obscured=3 ill-defined=4 spiculated=5 (nominal) Density: mass density high=1 iso=2 low=3 fat-containing=4 (ordinal) Severity: benign=0 or malignant=1 (binominal) BI-RADS is an assesment of how confident the severity classification is; it is not a "predictive" attribute and so we will discard it. The age, shape, margin, and density attributes are the features that we will build our model with, and "severity" is the classification we will attempt to predict based on those attributes.

Although "shape" and "margin" are nominal data types, which sklearn typically doesn't deal with well, they are close enough to ordinal that we shouldn't just discard them. The "shape" for example is ordered increasingly from round to irregular.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
data_path = 'mammographic+mass'
col_names = ['BI-RADS', 'Age', 'Shape', 'Margin', 'Density', 'Severity']
df_data = pd.read_csv(os.path.join(data_path, 'mammographic_masses.data'), names=col_names, na_values='?')
df_data.head()

,BI-RADS,Age,Shape,Margin,Density,Severity
0,5.0,67.0,3.0,5.0,3.0,1
1,4.0,43.0,1.0,1.0,NaN,1
2,5.0,58.0,4.0,5.0,3.0,1
3,4.0,28.0,1.0,1.0,3.0,0
4,5.0,74.0,1.0,5.0,NaN,1


In [3]:
df_data.shape

(961, 6)

In [4]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 961 entries, 0 to 960
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   BI-RADS   959 non-null    float64
 1   Age       956 non-null    float64
 2   Shape     930 non-null    float64
 3   Margin    913 non-null    float64
 4   Density   885 non-null    float64
 5   Severity  961 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 45.2 KB


In [5]:
df_data.describe()

,BI-RADS,Age,Shape,Margin,Density,Severity
count,959.000000,956.000000,930.000000,913.000000,885.000000,961.000000
mean,4.348279,55.487448,2.721505,2.796276,2.910734,0.463059
std,1.783031,14.480131,1.242792,1.566546,0.380444,0.498893
min,0.000000,18.000000,1.000000,1.000000,1.000000,0.000000
25%,4.000000,45.000000,2.000000,1.000000,3.000000,0.000000
50%,4.000000,57.000000,3.000000,3.000000,3.000000,0.000000
75%,5.000000,66.000000,4.000000,4.000000,3.000000,1.000000
max,55.000000,96.000000,4.000000,5.000000,4.000000,1.000000


In [6]:
df_data = df_data.dropna()
df_data = df_data.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))

In [7]:
df_data.describe()

,BI-RADS,Age,Shape,Margin,Density,Severity
count,830.000000,830.000000,830.000000,830.000000,830.000000,830.000000
mean,0.079890,0.484384,0.593976,0.453313,0.638554,0.485542
std,0.034334,0.188100,0.414120,0.391794,0.116979,0.500092
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.072727,0.358974,0.333333,0.000000,0.666667,0.000000
50%,0.072727,0.500000,0.666667,0.500000,0.666667,0.000000
75%,0.090909,0.615385,1.000000,0.750000,0.666667,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
import tensorflow as tf
print(tf.__version__)


2.10.0


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop, Adam

from sklearn.model_selection import cross_val_score
from scikeras.wrappers import KerasClassifier

In [10]:
X = df_data[['Age', 'Shape', 'Margin', 'Density']].values
y = df_data['Severity'].values

In [11]:
print(X.shape, y.shape)

(830, 4) (830,)


In [12]:
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(X.shape[1],)))
model.add(Dense(16, activation='relu', input_shape=(X.shape[1],)))
model.add(Dense(1, activation='sigmoid'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                160       
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 705
Trainable params: 705
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [27]:
# estimator = KerasClassifier(model=model, epochs=100, verbose=0)
# cv_scores = cross_val_score(estimator, X, y, cv=10)
# cv_scores.mean()

In [20]:
import random
idxs = random.sample(list(range(len(X))), int(len(X)))

In [23]:
X_train = X[idxs[:int(len(X)*0.8)]]
y_train = y[idxs[:int(len(X)*0.8)]]
X_test = X[idxs[int(len(X)*0.8)]:]
y_test = y[idxs[int(len(X)*0.8)]:]

In [24]:
history = model.fit(X_train, y_train,
                    batch_size=10,
                    epochs=100,
                    verbose=2,
                    )

Epoch 1/100
67/67 - 4s - loss: 0.6784 - accuracy: 0.6581 - 4s/epoch - 59ms/step
Epoch 2/100
67/67 - 0s - loss: 0.6072 - accuracy: 0.7801 - 289ms/epoch - 4ms/step
Epoch 3/100
67/67 - 0s - loss: 0.5307 - accuracy: 0.7967 - 287ms/epoch - 4ms/step
Epoch 4/100
67/67 - 0s - loss: 0.4852 - accuracy: 0.7907 - 326ms/epoch - 5ms/step
Epoch 5/100
67/67 - 0s - loss: 0.4699 - accuracy: 0.7907 - 265ms/epoch - 4ms/step
Epoch 6/100
67/67 - 0s - loss: 0.4700 - accuracy: 0.7952 - 297ms/epoch - 4ms/step
Epoch 7/100
67/67 - 0s - loss: 0.4597 - accuracy: 0.8042 - 290ms/epoch - 4ms/step
Epoch 8/100
67/67 - 0s - loss: 0.4600 - accuracy: 0.8012 - 312ms/epoch - 5ms/step
Epoch 9/100
67/67 - 0s - loss: 0.4556 - accuracy: 0.7982 - 308ms/epoch - 5ms/step
Epoch 10/100
67/67 - 0s - loss: 0.4536 - accuracy: 0.8057 - 305ms/epoch - 5ms/step
Epoch 11/100
67/67 - 0s - loss: 0.4524 - accuracy: 0.8042 - 296ms/epoch - 4ms/step
Epoch 12/100
67/67 - 0s - loss: 0.4496 - accuracy: 0.8117 - 308ms/epoch - 5ms/step
Epoch 13/100
67

Epoch 100/100
67/67 - 0s - loss: 0.4168 - accuracy: 0.8268 - 265ms/epoch - 4ms/step


In [25]:
acc = model.evaluate(X_test, y_test)

24/24 [==============================] - 0s 4ms/step - loss: 0.4383 - accuracy: 0.8126


In [26]:
acc

[0.4382634162902832, 0.8125836849212646]